# Medical Treatment Cost Data Retrieval Tool

## 0.1 Intent

The cost of medical treatment is a 'hot topic' in the U.S. Topics such as whether medical treatment is fairly priced and equal access to quality medical treatment are frequent subjects of debate. In order to enable individuals researching the interactions between medical bills, quality of medical care, and income disparity to easily be able to access data pertinent to these subjects I have created a tool that can be used to easily retrieve data from multiple sources. The tool is presented in this noteboook.

## 0.2 Data Description

### **Inpatient Medical Cost Data:** [CMS.gov IPPS Summary](https://data.cms.gov/Medicare-Inpatient/Inpatient-Prospective-Payment-System-IPPS-Provider/97k6-zzx3)

Description: This dataset contains cost averages for inpatient medical procedures at healthcare providers across the U.S. 

- **Covered Charges**: Average amount of payments covered by insurance. 
- **Medicare Payments**: Average amount paid by Medicare for this procedure.
- **Total Payments**: Average of all payments including out-of-pocket, covered by insurance, and covered by Medicare for this procedure.


### **Hospital Quality Data**: [Medicare.gov Hopsital Ratings Data](https://www.kaggle.com/center-for-medicare-and-medicaid/hospital-ratings)

Description: The dataset contains ratings for hospitals and healthcare providers across the U.S. Both an overall rating for a hosptial and ratings for specific categories of care quality are included. Aside from all overall rating, all ratings are with respect to the national average and take one of the three values, "Above the national average", "Same as the national average", or "Below the national average" for the respective measure of quality. 

- **Hospital Overall Rating**: Composite quality rating 1-5.
- **Mortality**
- **Safety of Care**
- **Readmission**
- **Patient Experience**
- **Effectiveness of Care**
- **Timeliness of Care**
- **Efficient Use of Medical Imaging**

### **City Median Household Income Data**: [Census.gov Community Facts Dataset](https://www.kaggle.com/kwullum/fatal-police-shootings-in-the-us)

Description: The dataset includes median household income data for all reported cities in the U.S. Data is organized by city and state.

- **Median Income**

## 1. Initialize and Configure SQL Database

A SQLite database is created and configured to hold the data outlined in 0.2. This will enable SQL operations to be peformed on the data.

### 1.1 Initialize SQLite Database

In [1]:
import sqlite3
from sqlite3 import Error
        
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
        return(conn)
    except Error as e:
        print(e)
        
conn = create_connection('data.db')

2.6.0


### 1.2 Create Database Tables

In [2]:
c = conn.cursor()

# Create inpatient charges table

c.execute("""CREATE TABLE IF NOT EXISTS INPATIENT (DRG nvarchar(50), Provider_ID int, Provider_Name nvarchar(50),
          Provider_Address nvarchar(50), Provider_City nvarchar(50), Provider_State nvarchar(50), Provider_Zip int, 
           Discharges int, Avg_Covered_Charges int, Avg_Total_Payments int, Avg_Medicare_Payments int)""")

# Create hospital quality table

c.execute("""CREATE TABLE IF NOT EXISTS QUALITY (Provider_ID int, Provider_Name nvarchar(50), Provider_Address nvarchar(50),
            Provider_City nvarchar(50), Provider_State nvarchar(50), Provider_Zip int, Hospital_Type nvarchar(50),
            Hospital_Ownership nvarchar(50), Emergency_Services nvarchar(50), ERH nvarchar(50), Rating int,
            Mortality nvarchar(50), Safety nvarchar(50), Readmission nvarchar(50), Patient_Experience nvarchar(50),
            Effectiveness nvarchar(50), Timeliness nvarchar(50), Efficient_Imaging nvarchar(50))""")

# Create income data table

c.execute("""CREATE TABLE IF NOT EXISTS INCOME (State nvarchar(50),City nvarchar(50),Median_Income int)""")

## 2. Populate Database

Data files are next loaded into Pandas dataframes and data columns are inserted into respective SQL tables and columns.

### 2.1 Load Data Files

In [3]:
import pandas as pd

# Load data files containing inpatient medical charge data, facility quality data, income data, and rent data
inpatient = pd.read_csv("InpatientCharges.csv")
quality = pd.read_csv("Quality.csv")
income = pd.read_csv("Income.csv")

### 2.2 Insert Dataframes into Tables

In [4]:
# Insert inpatient charges data into table
for row in inpatient.itertuples():
    
    c.execute('''
                INSERT INTO INPATIENT (DRG,Provider_ID,Provider_Name,Provider_Address,Provider_City,Provider_State,
                Provider_Zip, Discharges, Avg_Covered_Charges, Avg_Total_Payments, Avg_Medicare_Payments)
                VALUES (?,?,?,?,?,?,?,?,?,?,?)
                ''',
                [row._1, 
                row._2,
                row._3,
                row._4,
                row._5,
                row._6,
                row._7,
                row._9,
                row._10,
                row._11,
                row._12]
                )
    
    
# Insert hospital quality data into table

for row in quality.itertuples():
    c.execute('''
                INSERT INTO QUALITY (Provider_ID,Provider_Name,Provider_Address,Provider_City,Provider_State,
                Provider_Zip, Hospital_Type, Hospital_Ownership, Emergency_Services, ERH, Rating, Mortality, Safety,
                Readmission, Patient_Experience, Effectiveness, Timeliness, Efficient_Imaging)
                VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
                ''',
                [row._1,
                row._2,
                row._3,
                row._4,
                row._5,
                row._6,
                row._9,
                row._10,
                row._11,
                row._12,
                row._13,
                row._15,
                row._17,
                row._19,
                row._21,
                row._23,
                row._25,
                row._27
                ]
                )
    
# Insert income data into table
for row in income.itertuples():
    c.execute('''
                INSERT INTO INCOME (State, City, Median_Income)
                VALUES (?,?,?)
                ''',
                [row._1,
                 row._2,
                 row._3
                ]
                )
    
conn.commit()

## 3. Create Joined Table

Two inner joins are then performed within the database. The first retrieves a table joining corresponding medical care providers located in the INPATIENT and QUALITY tables, and the second adds to this table the corresponding values for median income of the city a provider is located in.

In [5]:
# Join Inpatient and Quality tables

c.execute('''CREATE TABLE INPATIENT_QUALITY AS 
SELECT *
FROM INPATIENT INNER JOIN QUALITY
ON INPATIENT.Provider_Address = QUALITY.Provider_Address''')

# Join Income table

c.execute('''CREATE TABLE INPATIENT_QUALITY_INCOME AS 
            SELECT *
            FROM INPATIENT_QUALITY INNER JOIN INCOME
            ON INPATIENT_QUALITY.Provider_State = INCOME.State AND INPATIENT_QUALITY.Provider_City = INCOME.City''')

conn.commit()

## 4. Create Data Retrieval Methods

In various instances it will be useful to examine the data on a provider-level (for example, when examining whether increased costs are associated with improved treatment outcome irrespective of geographical location) or on a city-level (examining whether areas with lower income receive lower quality medical care. Methods are therefore provided to retrieve averages for both of these groupings.

### 4.1 Retrieve Averages by Provider

In [6]:
def retrieveProviders():
    
    conn = create_connection('data.db')
    
    # Get dataframe
    
    df = pd.read_sql_query("SELECT * FROM INPATIENT_QUALITY_INCOME", conn)
    
    df.columns= df.columns.str.lower()
   
    # Ordinal quality variables are numerically coded before averaging
    
    replace_map = {
    'Below the national average':-1,
    'Same as the national average':0,
    'Above the national average':1
    }
    
    df = df.replace(replace_map)
    
    providers = df.groupby('provider_id').mean()
    
    df = df[['provider_id','state','city']]
    
    # remove duplicates
    
    df = df.drop_duplicates('provider_id')
    
    providers = providers.join(df.set_index('provider_id'))
    
    providers = providers.drop(['discharges','provider_id:1','provider_zip:1'],axis=1)
    
    return providers

In [18]:
providers = retrieveProviders()

2.6.0


In [19]:
providers

,provider_zip,avg_covered_charges,avg_total_payments,avg_medicare_payments,rating,mortality,safety,readmission,patient_experience,effectiveness,timeliness,efficient_imaging,median_income,state,city
provider_id,,,,,,,,,,,,,,,
10001,36301.0,35247.028152,8749.025109,7678.214348,3.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,42426.0,AL,DOTHAN
10005,35957.0,16451.092041,6812.131224,5793.631429,3.0,-1.0,0.0,1.0,0.0,0.0,1.0,-1.0,35974.0,AL,BOAZ
10006,35631.0,36942.357442,8197.237907,7145.959535,2.0,-1.0,0.0,0.0,-1.0,0.0,1.0,0.0,35731.0,AL,FLORENCE
10007,36467.0,12079.536818,4860.829091,4047.025455,2.0,0.0,NaN,-1.0,0.0,-1.0,1.0,NaN,34695.0,AL,OPP
10008,36049.0,16148.752222,5898.136667,4963.547778,3.0,0.0,NaN,0.0,NaN,0.0,1.0,NaN,29844.0,AL,LUVERNE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670071,76012.0,26435.206667,5078.561667,3945.660000,4.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,53326.0,TX,ARLINGTON
670073,75001.0,26091.035000,12670.465000,9176.107500,4.0,NaN,0.0,NaN,1.0,0.0,NaN,NaN,67695.0,TX,ADDISON
670075,77070.0,33840.385000,5255.138333,4116.481667,2.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,NaN,46187.0,TX,HOUSTON


### 4.2 Retrieve Averages by City

In [7]:
def retrieveCityData():
    
    providers = retrieveProviders()
    
    city_data = providers.groupby(['city','state']).mean()
    
    city_data = city_data.drop('provider_zip',axis=1)
    
    return city_data

In [20]:
city_data = retrieveCityData()

2.6.0


In [21]:
city_data

,,avg_covered_charges,avg_total_payments,avg_medicare_payments,rating,mortality,safety,readmission,patient_experience,effectiveness,timeliness,efficient_imaging,median_income
city,state,,,,,,,,,,,,
ABBEVILLE,LA,17186.322222,6663.217222,5936.423333,3.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,28158.0
ABERDEEN,WA,25956.866731,9281.255577,8368.124808,2.0,-1.0,-1.0,0.0,-1.0,0.0,0.0,0.0,40958.0
ABILENE,TX,49854.589640,8969.395537,7605.974107,3.5,-0.5,1.0,1.0,-0.5,0.0,-0.5,0.0,43189.0
ABINGDON,VA,28453.788889,7173.921270,6156.648571,3.0,0.0,1.0,-1.0,-1.0,0.0,0.0,1.0,39405.0
ADA,OK,9711.730000,8086.926667,7238.195556,3.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,35330.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZACHARY,LA,27039.455758,6374.074242,5473.054545,2.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,-1.0,74274.0
ZANESVILLE,OH,21499.367708,8932.169271,7632.901354,4.0,1.0,1.0,0.0,-1.0,1.0,1.0,0.0,26268.0
ZEELAND,MI,15800.856364,6736.034545,5499.131818,4.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,40733.0


## 5. Retrieve Correlation Matrices

Correlation matricies allow for quick visualization of related factors. Methods are provided to view correlations for both the provider and city groupings.

### 5.1 Retrieve Provider Correlation Matrix

In [22]:
def getProviderCorrelation():
    
    providers = retrieveProviders()
    providers = providers.drop(['provider_zip'],axis=1)
    corr = providers.corr()
    
    return corr

### 5.2 Retrieve City Correlation Matrix

In [23]:
def getCityCorrelation():

    city_data = retrieveCityData()
    corr = city_data.corr()
    
    return corr

### 5.3 View Correlation Matricies

In [24]:
p_corr = getProviderCorrelation()
c_corr = getCityCorrelation()

2.6.0
2.6.0


In [25]:
p_corr

,avg_covered_charges,avg_total_payments,avg_medicare_payments,rating,mortality,safety,readmission,patient_experience,effectiveness,timeliness,efficient_imaging,median_income
avg_covered_charges,1.000000,0.602709,0.586839,-0.128295,0.109408,-0.021446,-0.089086,-0.189791,0.067913,-0.357620,0.057586,0.286007
avg_total_payments,0.602709,1.000000,0.982984,-0.139341,0.150363,-0.130097,-0.083526,-0.072221,0.033835,-0.428133,0.104441,0.261670
avg_medicare_payments,0.586839,0.982984,1.000000,-0.165941,0.149946,-0.131112,-0.102619,-0.112351,0.024143,-0.434272,0.093558,0.235561
rating,-0.128295,-0.139341,-0.165941,1.000000,0.243683,0.431113,0.518761,0.585715,0.145254,0.268249,0.036897,0.095015
mortality,0.109408,0.150363,0.149946,0.243683,1.000000,-0.011324,-0.095324,0.018251,0.009100,-0.100135,-0.006652,0.162710
safety,-0.021446,-0.130097,-0.131112,0.431113,-0.011324,1.000000,0.042338,0.093348,0.050079,0.086221,-0.021894,-0.007980
readmission,-0.089086,-0.083526,-0.102619,0.518761,-0.095324,0.042338,1.000000,0.258456,0.066186,0.135784,-0.002031,0.048998
patient_experience,-0.189791,-0.072221,-0.112351,0.585715,0.018251,0.093348,0.258456,1.000000,0.109228,0.298733,0.002546,0.053772
effectiveness,0.067913,0.033835,0.024143,0.145254,0.009100,0.050079,0.066186,0.109228,1.000000,0.047155,0.052430,0.040258
timeliness,-0.357620,-0.428133,-0.434272,0.268249,-0.100135,0.086221,0.135784,0.298733,0.047155,1.000000,-0.131163,-0.159488


In [26]:
c_corr

,avg_covered_charges,avg_total_payments,avg_medicare_payments,rating,mortality,safety,readmission,patient_experience,effectiveness,timeliness,efficient_imaging,median_income
avg_covered_charges,1.000000,0.611990,0.600322,-0.127679,0.097560,-0.000689,-0.057322,-0.216648,0.045837,-0.388879,0.058456,0.333218
avg_total_payments,0.611990,1.000000,0.988267,-0.083794,0.112163,-0.087847,-0.000126,-0.088854,0.022504,-0.414186,0.123045,0.322467
avg_medicare_payments,0.600322,0.988267,1.000000,-0.106100,0.112972,-0.087840,-0.018879,-0.115137,0.012728,-0.415638,0.108511,0.285985
rating,-0.127679,-0.083794,-0.106100,1.000000,0.233803,0.389677,0.528692,0.574332,0.129982,0.230318,0.057559,0.105274
mortality,0.097560,0.112163,0.112972,0.233803,1.000000,-0.009122,-0.099596,-0.007538,-0.014289,-0.094537,0.001841,0.195630
safety,-0.000689,-0.087847,-0.087840,0.389677,-0.009122,1.000000,0.029469,0.047455,0.043973,0.048303,-0.000375,-0.020074
readmission,-0.057322,-0.000126,-0.018879,0.528692,-0.099596,0.029469,1.000000,0.270551,0.076120,0.114992,-0.006157,0.048541
patient_experience,-0.216648,-0.088854,-0.115137,0.574332,-0.007538,0.047455,0.270551,1.000000,0.092948,0.292644,0.015356,0.063884
effectiveness,0.045837,0.022504,0.012728,0.129982,-0.014289,0.043973,0.076120,0.092948,1.000000,0.040266,0.057571,0.063733
timeliness,-0.388879,-0.414186,-0.415638,0.230318,-0.094537,0.048303,0.114992,0.292644,0.040266,1.000000,-0.159592,-0.206469
